In [1]:
import pandas as pd
import networkx as nx

In [2]:
# load dataset
business = pd.read_csv('./business.csv')
user = pd.read_csv('./user.csv')
review = pd.read_csv('./review.csv')

1단계 yelp data 중 성균관대학교 주변 음식점과 유사한 환경 조건의 데이터 pre-processing

- yelp data 중 총 business 150346 rows 중 Restaurant 또는 Food 관련 business 64629row 추출
- 이 중 가장 많은 business가 등록된 도시인 Philadelphia 7076 row 내에서 University City(대학 주변 상권)에 해당하는 business 508 row 추출
- 추출된 business에 해당되는 review 총 4002 row 추출
- 추출된 review를 작성한 user 총 2792 row 추출

In [3]:
'''
     [business Table : restaurant information - 필라델피아 university district 지역]
     
1. business_id : business_id
2. name : restaurant_name
3. address : address
4. city : Philadelphia
5. stars : 레스토랑에 대한 평균 평점
6. review_count : 해당 레스토랑에 대한 리뷰 개수


'''
business = business[['business_id', 'name','address','city','stars','review_count']]
business

,business_id,name,address,city,stars,review_count
0,ppFCk9aQkM338Rgwpl2F5A,Wawa,3604 Chestnut St,Philadelphia,3.0,56
1,bqmX882_6AxVA_uHEQ70sQ,Shanghai Station,4002 Spruce St,Philadelphia,4.0,46
2,RrqRin8BYGrTlJbCDwtLzA,Wawa,3744 Spruce St,Philadelphia,3.5,50
3,12kD46cnLyMXUQTCI2N0ug,P&P Grocery,4307 Locust St,Philadelphia,4.5,13
4,N1ZXhAVzOzbyCBPEXTaUUw,Tandoor Authentic Indian Restaurant,106 S 40th St,Philadelphia,3.5,148
...,...,...,...,...,...,...
503,pM1M2A14mNXz48KCcQtz2w,Starbucks,4140 Woodland Ave,Philadelphia,3.0,17
504,662o6Z3l7o0PDi8Y-88oWQ,Creative Cafe,3711 Market St,Philadelphia,4.0,14
505,OfRbGvHn-nAnQMqNxiIpcA,Grilly Cheese,3306 Arch St,Philadelphia,3.0,5
506,VZbkSeZtFynEascotq7ExA,Ali Baba Magic Food,37TH And Walnut,Philadelphia,4.0,8


In [4]:
'''
     [User Table : user information]
     
1. user_id : user_id
2. name : user_name
3. review_count : 리뷰를 남긴 개수
4. useful : 해당 유저가 받은 'useful' 추천 개수
5. fans : 해당 유저를 팔로우 하는 사람의 수 

'''
user = user[['user_id','name','review_count','useful','fans']]
user

,user_id,name,review_count,useful,fans
0,FlhZYUEhfpfm0Ynxacftwg,Kate,94,124,6
1,NWFqEfs5Z9wUmCeVfTTDAA,David,16,11,1
2,LfleZY423IgY6tHULVXGcw,Baeth,25,51,2
3,ojxS1v-8nUvEEx4DlsPQrA,Kristin,655,621,34
4,FlXBpK_YZxLo27jcMdII1w,Mallory,1158,3875,102
...,...,...,...,...,...
2787,Q9PK-ZrG-Mhr_Nwx2NXW9g,John,12,13,0
2788,NBF073P4GpQgrEFOO8LGMQ,Michael,1,0,0
2789,XXJ75Ya_hxdcAEPlk0z3yA,Maimon,3,0,0
2790,sr_LDk6CIwHlXxhrSWKHkA,Chris,1,0,0


In [5]:
'''
     [Review Table : restaurant에 대한 user의 리뷰]
     
1. review_id : review_id
2. user_id : user_id
3. business_id : business_id
4. stars : 별점점
5. useful : useful 추천을 받은 개수
6. date : 리뷰 날짜

'''
review=review[['review_id','user_id','business_id','stars','useful','date']]
review

,review_id,user_id,business_id,stars,useful,date
0,Hov6E6epvJRM6gjDIKLLHw,sIH5HbEwhbxt-4waYoDUBw,RXItVN-qOBgbp_vBalAkOw,4.0,1,2019-12-02 13:46:32
1,OJCHbiPhOvh3jWFTVC0hcA,LV-7j98okmkBOeE-EA3RwQ,-CRjqZ4GxZ6lIa_qMj7-yw,5.0,0,2020-06-28 21:07:14
2,TZzka79A588bOiOlHrKz-A,gc1YP39bqjYFP6tMlkcSZQ,MMRRS6YhVRx_iN5-JhMRYg,3.0,0,2020-04-10 19:40:49
3,1cOVhOGu6ES8vD4Kwf2L7A,Zq31drx-JM2R1MKQg8uJQw,MMRRS6YhVRx_iN5-JhMRYg,4.0,2,2020-01-23 10:20:21
4,zi_tcs-VnFXJPFSQrWXnUg,TJ8Hawan8jDIZHS7AAZJwg,B-DiQpcSTJ7oMMnwzbAGTQ,4.0,0,2020-01-25 01:22:51
...,...,...,...,...,...,...
3997,5gZ8vC-xtzcF5ar7FpTNvg,G5OLGMmgH1idDBMwKbiSSA,zf10JmXNZFljGl3IgWW-Lg,5.0,1,2021-09-20 00:24:17
3998,wTjDM7o5_bsn3oMUqaMvOg,URrgQD93DohctUfRqjYWAA,jL4fYp0j8Jn31BiYptb10g,3.0,3,2020-12-05 00:24:19
3999,gQe6iUZMkCxIjZDNrY0nHg,UF93ng54bB4981krwCMmKw,dmyPzMRNgTIOBRT11NFcvg,4.0,1,2020-01-25 21:16:36
4000,u_pQ6h4bUiH-2J6UB9HysQ,6knga-vNLcCxdQmhevWjTA,irolxiYw4pOCM9gGDOhnsw,1.0,1,2021-06-19 00:53:57


In [6]:
review = pd.merge(review, user[['user_id','name']], how='left', left_on='user_id', right_on='user_id')

In [7]:
review = review[['review_id', 'name','business_id', 'stars', 'useful', 'date']]
review.columns = ['review_id', 'user_name','business_id', 'stars', 'useful', 'date']

In [8]:
review

,review_id,user_name,business_id,stars,useful,date
0,Hov6E6epvJRM6gjDIKLLHw,Casey,RXItVN-qOBgbp_vBalAkOw,4.0,1,2019-12-02 13:46:32
1,OJCHbiPhOvh3jWFTVC0hcA,Cailey,-CRjqZ4GxZ6lIa_qMj7-yw,5.0,0,2020-06-28 21:07:14
2,TZzka79A588bOiOlHrKz-A,Rubaiyet,MMRRS6YhVRx_iN5-JhMRYg,3.0,0,2020-04-10 19:40:49
3,1cOVhOGu6ES8vD4Kwf2L7A,Long,MMRRS6YhVRx_iN5-JhMRYg,4.0,2,2020-01-23 10:20:21
4,zi_tcs-VnFXJPFSQrWXnUg,Max,B-DiQpcSTJ7oMMnwzbAGTQ,4.0,0,2020-01-25 01:22:51
...,...,...,...,...,...,...
3997,5gZ8vC-xtzcF5ar7FpTNvg,Angel,zf10JmXNZFljGl3IgWW-Lg,5.0,1,2021-09-20 00:24:17
3998,wTjDM7o5_bsn3oMUqaMvOg,Konyin,jL4fYp0j8Jn31BiYptb10g,3.0,3,2020-12-05 00:24:19
3999,gQe6iUZMkCxIjZDNrY0nHg,Kaliah,dmyPzMRNgTIOBRT11NFcvg,4.0,1,2020-01-25 21:16:36
4000,u_pQ6h4bUiH-2J6UB9HysQ,Jack,irolxiYw4pOCM9gGDOhnsw,1.0,1,2021-06-19 00:53:57


In [9]:
review = pd.merge(review, business[['business_id', 'name']], how='left', left_on='business_id', right_on='business_id')

In [10]:
review

,review_id,user_name,business_id,stars,useful,date,name
0,Hov6E6epvJRM6gjDIKLLHw,Casey,RXItVN-qOBgbp_vBalAkOw,4.0,1,2019-12-02 13:46:32,Goldie
1,OJCHbiPhOvh3jWFTVC0hcA,Cailey,-CRjqZ4GxZ6lIa_qMj7-yw,5.0,0,2020-06-28 21:07:14,Spring Chinese Restaurant
2,TZzka79A588bOiOlHrKz-A,Rubaiyet,MMRRS6YhVRx_iN5-JhMRYg,3.0,0,2020-04-10 19:40:49,Han Dynasty
3,1cOVhOGu6ES8vD4Kwf2L7A,Long,MMRRS6YhVRx_iN5-JhMRYg,4.0,2,2020-01-23 10:20:21,Han Dynasty
4,zi_tcs-VnFXJPFSQrWXnUg,Max,B-DiQpcSTJ7oMMnwzbAGTQ,4.0,0,2020-01-25 01:22:51,Cucina Zapata
...,...,...,...,...,...,...,...
3997,5gZ8vC-xtzcF5ar7FpTNvg,Angel,zf10JmXNZFljGl3IgWW-Lg,5.0,1,2021-09-20 00:24:17,Mizu
3998,wTjDM7o5_bsn3oMUqaMvOg,Konyin,jL4fYp0j8Jn31BiYptb10g,3.0,3,2020-12-05 00:24:19,Woosa Bubble Tea
3999,gQe6iUZMkCxIjZDNrY0nHg,Kaliah,dmyPzMRNgTIOBRT11NFcvg,4.0,1,2020-01-25 21:16:36,Starbucks
4000,u_pQ6h4bUiH-2J6UB9HysQ,Jack,irolxiYw4pOCM9gGDOhnsw,1.0,1,2021-06-19 00:53:57,Boston Market


In [11]:
review = review[['review_id', 'name','user_name', 'stars', 'useful', 'date']]
review.columns = ['review_id', 'business_name','user_name', 'stars', 'useful', 'date']
review

,review_id,business_name,user_name,stars,useful,date
0,Hov6E6epvJRM6gjDIKLLHw,Goldie,Casey,4.0,1,2019-12-02 13:46:32
1,OJCHbiPhOvh3jWFTVC0hcA,Spring Chinese Restaurant,Cailey,5.0,0,2020-06-28 21:07:14
2,TZzka79A588bOiOlHrKz-A,Han Dynasty,Rubaiyet,3.0,0,2020-04-10 19:40:49
3,1cOVhOGu6ES8vD4Kwf2L7A,Han Dynasty,Long,4.0,2,2020-01-23 10:20:21
4,zi_tcs-VnFXJPFSQrWXnUg,Cucina Zapata,Max,4.0,0,2020-01-25 01:22:51
...,...,...,...,...,...,...
3997,5gZ8vC-xtzcF5ar7FpTNvg,Mizu,Angel,5.0,1,2021-09-20 00:24:17
3998,wTjDM7o5_bsn3oMUqaMvOg,Woosa Bubble Tea,Konyin,3.0,3,2020-12-05 00:24:19
3999,gQe6iUZMkCxIjZDNrY0nHg,Starbucks,Kaliah,4.0,1,2020-01-25 21:16:36
4000,u_pQ6h4bUiH-2J6UB9HysQ,Boston Market,Jack,1.0,1,2021-06-19 00:53:57


## 어떤 레스토랑이 많은 리뷰를 받았는가?

In [12]:
rest_cnt = review['business_name'].value_counts()
rest_cnt

White Dog Cafe                     143
Louie Louie                        111
Sabrina's Café                     105
Manakeesh Cafe Bakery & Grill      103
Han Dynasty                         84
                                  ... 
Pizza Rustica                        1
Traditional Szechuan Restaurant      1
Rana Halal Cuisine                   1
Nathan's Famous                      1
King Grill                           1
Name: business_name, Length: 258, dtype: int64

In [13]:
sum(list(rest_cnt >= 20))

60

### 리뷰를 20개 이상 받은 레스토랑 리스트 생성

In [14]:
target_rid = list(rest_cnt[list(rest_cnt>=20)].keys())
target_rid

['White Dog Cafe',
 'Louie Louie',
 "Sabrina's Café",
 'Manakeesh Cafe Bakery & Grill',
 'Han Dynasty',
 "Renata's Kitchen",
 'Don Barriga Mexican Grill',
 'Dim Sum House',
 "Bart's Bagels",
 'Haveli Virasat',
 'Walnut Street Cafe',
 "Beiler's Donuts",
 'Bonchon University City',
 'Abyssinia',
 'Terakawa Ramen',
 'Tacotacomexican',
 'LaTao HotPot',
 'Chengdu Famous Food',
 'Kabobeesh',
 'Distrito',
 'Baby Blues BBQ Philly',
 'Pho & Cafe Saigon',
 'Clarkville',
 'Quality Taste Jamaican Restaurant',
 'Landmark Americana Tap & Grill - University City',
 'NAM Vietnamese Kitchen',
 'Sang Kee Noodle House',
 'Ochatto',
 'Pattaya',
 'Coney Shack',
 'Greek Lady',
 'New Delhi Indian Restaurant',
 'Spread Bagelry',
 'Hadramout',
 'Yiro Yiro',
 'Boba King',
 "Koch's Deli",
 "Ed's Buffalo Wings & Pizza",
 'Tipsy Bistro',
 'Pelicana chicken',
 'El Taco',
 'Local 44',
 'Chili Szechuan',
 'Thai Singha House',
 'Grindcore House',
 'HipCityVeg - University City',
 "Abner's Cheesesteaks",
 'Starbucks',


In [15]:
print(len(target_rid))

60


### 60개의 타겟 레스토랑만 포함하는 filtered review table 생성

In [16]:
f_review = review[[i in target_rid for i in review['business_name']]].reset_index(drop=True)
f_review

,review_id,business_name,user_name,stars,useful,date
0,TZzka79A588bOiOlHrKz-A,Han Dynasty,Rubaiyet,3.0,0,2020-04-10 19:40:49
1,1cOVhOGu6ES8vD4Kwf2L7A,Han Dynasty,Long,4.0,2,2020-01-23 10:20:21
2,hJi8nhPnok5hvTw_vBSSWw,Han Dynasty,Amita,4.0,1,2019-12-15 21:36:04
3,Vx_hrWEhbvsgNnhDRzl6IA,HipCityVeg - University City,Renée,5.0,0,2020-04-13 23:25:59
4,lRCh4flj3Izykbt7EANb1w,Han Dynasty,AJ,5.0,2,2020-01-21 07:09:08
...,...,...,...,...,...,...
2512,A3M4rUcVDSaAi_F33H2rSg,Koreana,David,4.0,3,2021-09-01 17:55:07
2513,MqO6AtW0LoRuKMyq0BXBuQ,Chili Szechuan,Angie,4.0,3,2020-12-09 01:28:18
2514,tPcO3FOEHa5vbINcqOA9eQ,Koreana,Jacqueline,2.0,0,2021-08-28 00:17:59
2515,m63xpqYvhspWGcsOnm5SJw,Koreana,Cindy,5.0,0,2020-05-15 19:00:31


### filtered restaurant table

In [17]:
rid_list = list(set(f_review['business_name'])) 
f_rest = business[[i in rid_list for i in business['name']]]
f_rest

,business_id,name,address,city,stars,review_count
8,MMRRS6YhVRx_iN5-JhMRYg,Han Dynasty,3711 Market St,Philadelphia,4.0,783
22,C_EtrXTygRX5RTUOKtO6Dg,Copabanana,4000 Spruce St,Philadelphia,3.0,334
23,LnZvGYbqozanOSevcftnrw,HipCityVeg - University City,214 S 40th St,Philadelphia,4.0,314
61,YN4Kk751tmdvoarGo8z7_A,New Delhi Indian Restaurant,4004 Chestnut St,Philadelphia,4.0,514
76,x5GkG0oI_S107wLlgdBxVA,Spread Bagelry,3602 Chestnut St,Philadelphia,2.5,203
...,...,...,...,...,...,...
453,bdteJqgoj0OaALhj9RFtmw,Koreana,3801 Chestnut St,Philadelphia,3.5,363
466,HgxY_RnJFGqqSVO32nxS4A,Chili Szechuan,4205 Chestnut St,Philadelphia,4.0,147
479,3gVSrS4kffGGZT8oXHsIcw,Pod,3636 Sansom St,Philadelphia,3.5,797
480,dmyPzMRNgTIOBRT11NFcvg,Starbucks,1128 Walnut St,Philadelphia,4.0,11


### filtered user table

In [18]:
uid_list = list(set(f_review['user_name']))
f_user = user[[i in uid_list for i in user['name']]]
f_user

,user_id,name,review_count,useful,fans
0,FlhZYUEhfpfm0Ynxacftwg,Kate,94,124,6
1,NWFqEfs5Z9wUmCeVfTTDAA,David,16,11,1
2,LfleZY423IgY6tHULVXGcw,Baeth,25,51,2
3,ojxS1v-8nUvEEx4DlsPQrA,Kristin,655,621,34
4,FlXBpK_YZxLo27jcMdII1w,Mallory,1158,3875,102
...,...,...,...,...,...
2783,zGPbPANUG9mr9NmjuwjcXw,Lisa,1,1,0
2784,75C_NMRsOF3VWDjZj2YuOg,Cameron,6,3,0
2787,Q9PK-ZrG-Mhr_Nwx2NXW9g,John,12,13,0
2788,NBF073P4GpQgrEFOO8LGMQ,Michael,1,0,0


# Graph Build 
- 세 가지 데이터셋을 활용해 그래프를 정의
- 그래프는 NetworkX로 정의
- 그래프 구성
    1. Node : Restaurant, User (Heterogeneous Graph)
    2. Edge : User-Restaurant (User가 Restaurant 대해 리뷰를 남긴 경우 연결)
    3. Edge Type : **Undirected**

In [19]:
G = nx.Graph() 

## 정의한 그래프에 Node를 추가
- Restaurant 노드와 user 노드를 순서대로 추가
- 각각의 노드가 어떤 유형의 노드인지 구분하기 위해, node_type이라는 attribute를 추가

- Restaurant 노드의 node_id 및 attribute
    - Node_id : name
    - Attribute : stars, review_count, node_type='restaurant'

- User 노드의 node_id 및 attribute
    - Node_id : name
    - Attribute : review_count, useful, fans, node_type='user'

In [20]:
rest_node = []
for n, s, rc in zip(f_rest['name'], f_rest['stars'], f_rest['review_count']):
  rest_node.append((n, {"stars":s, "review_count":rc, "node_type":'restaurant'}))
G.add_nodes_from(rest_node)

In [21]:
for node in G.nodes(data=True):
  print(node)
  break

('Han Dynasty', {'stars': 4.0, 'review_count': 783, 'node_type': 'restaurant'})


In [22]:
user_node = []
for n, rc, u, f in zip(f_user['name'], f_user['review_count'], f_user['useful'], f_user['fans']):
  user_node.append((n, {"review_count":rc, "useful":u, "fans":f, "node_type":'user'}))
G.add_nodes_from(user_node)

In [23]:
for node in G.nodes(data=True):
  if node[0] == 'Kate':
    print(node)
    break

('Kate', {'review_count': 1, 'useful': 0, 'fans': 0, 'node_type': 'user'})


In [24]:
f_review

,review_id,business_name,user_name,stars,useful,date
0,TZzka79A588bOiOlHrKz-A,Han Dynasty,Rubaiyet,3.0,0,2020-04-10 19:40:49
1,1cOVhOGu6ES8vD4Kwf2L7A,Han Dynasty,Long,4.0,2,2020-01-23 10:20:21
2,hJi8nhPnok5hvTw_vBSSWw,Han Dynasty,Amita,4.0,1,2019-12-15 21:36:04
3,Vx_hrWEhbvsgNnhDRzl6IA,HipCityVeg - University City,Renée,5.0,0,2020-04-13 23:25:59
4,lRCh4flj3Izykbt7EANb1w,Han Dynasty,AJ,5.0,2,2020-01-21 07:09:08
...,...,...,...,...,...,...
2512,A3M4rUcVDSaAi_F33H2rSg,Koreana,David,4.0,3,2021-09-01 17:55:07
2513,MqO6AtW0LoRuKMyq0BXBuQ,Chili Szechuan,Angie,4.0,3,2020-12-09 01:28:18
2514,tPcO3FOEHa5vbINcqOA9eQ,Koreana,Jacqueline,2.0,0,2021-08-28 00:17:59
2515,m63xpqYvhspWGcsOnm5SJw,Koreana,Cindy,5.0,0,2020-05-15 19:00:31


In [25]:
#edge 추가
edges = []
for u, b, s in zip(f_review['user_name'], f_review['business_name'], f_review['stars']):
  edges.append((u,b, {"weight":s}))
G.add_edges_from(edges)

In [26]:
G.number_of_edges(), G.number_of_nodes()

(2399, 1268)

In [ ]:
#그래프 저장
nx.write_graphml_lxml(G, "yelp_graph_v2.graphml")

In [30]:
from networkx.algorithms import bipartite

In [32]:
bc = bipartite.clustering(G)

In [51]:
sum(list(bc.values()))/len(list(bc.values()))

0.3971006554672986

In [83]:
rest_node = list(G.nodes)[:60]

In [90]:
bc_key = bipartite.clustering(G).keys()

In [91]:
bc_value = bipartite.clustering(G).values()

In [92]:
rest_bc_values = []
for i, j in zip(bc_key, bc_value):
  if i in rest_node:
    rest_bc_values.append(j)
len(rest_bc_values)

60

In [93]:
sum(rest_bc_values)/len(rest_bc_values)

0.03307468717451624

In [ ]:
# n : The number of nodes 
# p : Probability for edge creation 
n = 20 #노드 수
p = 0.5 #랜덤 연결 확률
random_G = nx.erdos_renyi_graph(n, p, seed=None, directed=False)

In [94]:
G.number_of_edges(), G.number_of_nodes()

(2399, 1268)

In [106]:
2399/(60*1268)

0.03153259726603575

In [96]:
2399/1208

1.9859271523178808

In [107]:
br = bipartite.random_graph(60, 1208, 0.03, seed=None, directed=False)

In [108]:
br.nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 

In [109]:
brc = bipartite.clustering(br)

In [110]:
brc

{0: 0.025650141816800195,
 1: 0.02210523289151093,
 2: 0.022090045674061348,
 3: 0.023649008715243464,
 4: 0.02524167638053748,
 5: 0.025905828681959092,
 6: 0.022573904114186343,
 7: 0.02244139042310318,
 8: 0.022773935349933695,
 9: 0.024483079531775715,
 10: 0.02349428745136365,
 11: 0.019901748126653768,
 12: 0.022315386681953795,
 13: 0.0247341919404532,
 14: 0.021524462610033556,
 15: 0.020964628571967337,
 16: 0.023563847475126746,
 17: 0.022496721026273026,
 18: 0.0232556472587145,
 19: 0.026503351104986677,
 20: 0.026491865469380038,
 21: 0.023100877226511437,
 22: 0.021200698243932534,
 23: 0.021593713853255335,
 24: 0.021189045253626818,
 25: 0.023304802907321523,
 26: 0.02295851900228271,
 27: 0.021439967826704567,
 28: 0.025376247705697335,
 29: 0.023728802056987873,
 30: 0.023302507532935276,
 31: 0.030000383531558296,
 32: 0.02088064817303976,
 33: 0.020600643369110737,
 34: 0.02527943290579531,
 35: 0.023336893448611725,
 36: 0.017863658713973392,
 37: 0.025206147143337

In [111]:
brc_keys = brc.keys()
brc_values = brc.values()

In [114]:
rest_brc_values = []
for i, j in zip(brc_keys, brc_values):
  if i <60:
    rest_brc_values.append(j)
len(rest_brc_values)

60

In [115]:
sum(rest_brc_values)/len(rest_brc_values)

0.023376689235062934